<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/cudf_cu_cat_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
# !pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
# !pip install -U git+https://github.com/graphistry/pygraphistry.git@cudf_cat_alex
!pip install --user --no-deps git+https://github.com/graphistry/cu-cat.git@cudf-cat #@0.01.0

# cu-cat/tree/cudf-cat/cuCat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/graphistry/cu-cat.git (to revision cudf-cat) to /tmp/pip-req-build-rc98w5ip
  Running command git clone --filter=blob:none --quiet https://github.com/graphistry/cu-cat.git /tmp/pip-req-build-rc98w5ip
  Running command git checkout -b cudf-cat --track origin/cudf-cat
  Switched to a new branch 'cudf-cat'
  Branch 'cudf-cat' set up to track remote branch 'cudf-cat' from 'origin'.
  Resolved https://github.com/graphistry/cu-cat.git to commit 0034eb570b7f6e0a77003c1d89a61bd706282261
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
import cuml,cudf
cuml.__version__

'22.10.00a+119.gb30ef85c1'

In [4]:
!echo "0.01.0" > VERSION.txt
!cp VERSION.txt /root/.local/lib/python3.9/site-packages/cu_cat/

In [ ]:
# from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

In [6]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)


In [7]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key
graphistry.__version__


'0.28.6+290.gc583814'

In [8]:
import logging
logger = logging.getLogger()


### redteam

In [9]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))
tdf.RED.sum()

749.0

In [19]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(tdf, 'node')

g5a = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
             remove_node_column=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         10070594 function calls (9911042 primitive calls) in 73.639 seconds

   Ordered by: internal time
   List reduced from 2272 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8   22.379    2.797   28.023    3.503 /root/.local/lib/python3.9/site-packages/cu_cat/_gap_encoder.py:1012(_multiplicative_update_h_smallfast)
        3   16.340    5.447   16.340    5.447 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
42818/42590    8.018    0.000   24.475    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
    39524    6.540    0.000    7.999    0.000 /usr/local/lib/python3.9/dist-packages/pandas/core/missing.py:61(mask_missing)
     4931    5.034    0.001    5.091    0.001 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
    39569    3.687    0.000    3.687    0.000 {method 'copy' of 'numpy.ndarray' objects}
    79499    2.046    0.000    2.046   

In [20]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(cudf.from_pandas(tdf), 'node')

g5b = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
             remove_node_column=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cudf_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         5074395 function calls (4992653 primitive calls) in 48.107 seconds

   Ordered by: internal time
   List reduced from 2337 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8   22.795    2.849   27.980    3.498 /root/.local/lib/python3.9/site-packages/cu_cat/_gap_encoder.py:1012(_multiplicative_update_h_smallfast)
        3   14.617    4.872   14.618    4.873 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
     4909    4.685    0.001    4.743    0.001 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
        7    1.017    0.145    3.119    0.446 /usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:1258(_count_vocab)
    63053    0.842    0.000    1.176    0.000 /usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:275(_char_ngrams)
      240    0.516    0.002    0.702    0.003 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/

In [21]:
# !pip install dirty_cat

In [22]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes((tdf), 'node')

g5c = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            #  remove_node_column=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirty_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         28856834 function calls (28697263 primitive calls) in 95.215 seconds

   Ordered by: internal time
   List reduced from 2174 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
3004535/3004296   18.719    0.000   35.050    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        3   16.110    5.370   16.110    5.370 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
     1054   14.352    0.014   40.516    0.038 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
    39524    5.489    0.000    7.025    0.000 /usr/local/lib/python3.9/dist-packages/pandas/core/missing.py:61(mask_missing)
    39616    3.459    0.000    3.459    0.000 {method 'copy' of 'numpy.ndarray' objects}
      775    3.441    0.004   10.193    0.013 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
   272946    3.164    

### ask HN

In [23]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# df = askHN.sample(1000,replace=True) # set smaller if you want to test a minibatch 
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [ ]:
g = graphistry.nodes(cudf.from_pandas(askHN))
pr = cProfile.Profile()
pr.enable()

g1 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g1a = g1.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cudf-cu_cat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat


In [ ]:
g1a = g1.umap(engine='cuml')

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()

g2 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g2a = g2.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cu_cat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g2a = g2.umap(engine='cuml')

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


g3 = g.featurize(feature_engine='dirty_cat',memoize=False,remove_node_column=False)
# g3a = g3.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
# g1a.plot()

In [ ]:
# g1b.plot()

### CTU-13 malware dataset

In [ ]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



In [ ]:
# lets umap the data
g = graphistry.edges(cudf.from_pandas(edf), src, dst)

pr = cProfile.Profile()
pr.enable()
g6b = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cudf-cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


In [ ]:
# lets umap the data
g = graphistry.edges(edf, src, dst)

pr = cProfile.Profile()
pr.enable()
g6a = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


In [ ]:
### lets umap the data
g = graphistry.edges(edf, src, dst)
pr = cProfile.Profile()
pr.enable()
g6c = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='dirty_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


### 20newsgroups

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 1000
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [ ]:
g = graphistry.nodes(cudf.from_pandas(news))
pr = cProfile.Profile()
pr.enable()


g8 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g8b = g8.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cudf-cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


g7 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g7a = g7.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()

g9 = g.featurize(feature_engine='dirty_cat',memoize=False,remove_node_column=False)
g9c = g9.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

### winlogs

In [ ]:
winlogs=pd.read_parquet('part.88 (1).parquet')
winlogs=winlogs.sample(100000,replace=False).iloc[:,:10]

g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g10 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g10a = g10.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_cu_cat_100k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

g = graphistry.nodes(cudf.from_pandas(winlogs))
pr = cProfile.Profile()
pr.enable()

g11 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g11b = g11.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_cudf-cu_cat_100k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()


g12 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g12c = g12.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_dirtycat_100k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
winlogs=pd.read_parquet('part.88 (1).parquet')
winlogs=winlogs.sample(1000000,replace=False).iloc[:,:10]

# g = graphistry.nodes(winlogs)
# pr = cProfile.Profile()
# pr.enable()

# g13 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g13a = g13.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize_cu_cat_1m.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

g = graphistry.nodes(cudf.from_pandas(winlogs))
pr = cProfile.Profile()
pr.enable()

g14 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g14b = g14.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_cudf-cu_cat_1m.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)
    

# g = graphistry.nodes(winlogs)
# pr = cProfile.Profile()
# pr.enable()

# g15 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g15b = g15.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)


# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize_dirtycat_1m.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

In [ ]:
stopppp